<a href="https://colab.research.google.com/github/PauDorMor/DPIEM/blob/main/PI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DPIEM
En el siguiente codigo se muestra y se trata de explicar el funcionamiento completo del codigo, el cual tiene la finalidad de poder clasificar e identificar EM disponibles en un espacio hospitalario

#Importe de librerias

In [ ]:
from roboflow import Roboflow
import os
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tqdm import tqdm
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
import pandas as pd
import joblib
from scipy.special import softmax


In [ ]:
pip install roboflow

#Extracción de imagenes y carpetas de la base de datos

In [ ]:
# Import the Roboflow library
from roboflow import Roboflow
import os

# NOTE: You will need to replace these with your actual Roboflow API key and project details.
# You can find your API key in your Roboflow account settings.
# The project details (workspace, project, version) are available on your dataset's page.


ROBOFLOW_API_KEY = "ysXFekfRklPYfXPY5kQa"
ROBOFLOW_WORKSPACE = "pau-dm-quidt"
ROBOFLOW_PROJECT = "medical-equipment-detection-t5ltj"
ROBOFLOW_VERSION = 3 # or the version number you want to download

# Initialize Roboflow
rf = Roboflow(api_key="ysXFekfRklPYfXPY5kQa")
project = rf.workspace("pau-dm-quidt").project("medical-equipment-detection-t5ltj")

# Download the dataset
# Specify the format (e.g., 'yolov5', 'tensorflow', 'voc', 'coco') that matches your needs.
# For example, if you want to use it with TensorFlow/Keras, 'tensorflow' might be appropriate.
dataset = project.version(ROBOFLOW_VERSION).download("tfrecord")

print(f"Dataset downloaded to: {dataset.location}")

# You can then access the dataset details:
# print(dataset.splits)
# print(dataset.path)

loading Roboflow workspace...
loading Roboflow project...
Dataset downloaded to: /content/medical-equipment-detection-3


In [ ]:

# Assuming 'dataset.location' holds the path to your downloaded Roboflow dataset
roboflow_dataset_path = dataset.location

print(f"Contents of the Roboflow dataset directory '{roboflow_dataset_path}':")
for root, dirs, files in os.walk(roboflow_dataset_path):
    level = root.replace(roboflow_dataset_path, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        print(f'{subindent}{f}')

Contents of the Roboflow dataset directory '/content/medical-equipment-detection-3':
medical-equipment-detection-3/
    README.dataset.txt
    README.roboflow.txt
    valid/
        medical-euipment-detection.tfrecord
        medical-euipment-detection_label_map.pbtxt
    train/
        medical-euipment-detection.tfrecord
        medical-euipment-detection_label_map.pbtxt
    test/
        medical-euipment-detection.tfrecord
        medical-euipment-detection_label_map.pbtxt


In [ ]:
# Assuming Ruta_general is already defined from previous cells
Ruta_general = '/content/medical-equipment-detection-3'

# Path to the label map file (using the train split as an ejemplo)
train_label_map_dir = os.path.join(Ruta_general, 'train')
label_map_filename = None

# Dynamically find the .pbtxt file in the directory
for file in os.listdir(train_label_map_dir):
    if file.endswith('_label_map.pbtxt'):
        label_map_filename = file
        break

if label_map_filename is None:
    raise FileNotFoundError(f"Could not find a label map file in {train_label_map_dir}")

label_map_path = os.path.join(train_label_map_dir, label_map_filename)

# Store all item names as they appear, and the id-to-name mapping
raw_class_names = []
label_id_to_raw_name = {}
with open(label_map_path, 'r') as f:
    content = f.read()
    # More robust regex to find id and display_name within each item block
    items_blocks = re.findall(r'item\s*{([^}]*)}', content)
    for block in items_blocks:
        id_match = re.search(r'id:\s*(\d+)', block)
        name_match = re.search(r'display_name:\s*"([^"]*)"', block)
        if id_match and name_match:
            item_id = id_match.group(1)
            item_name = name_match.group(1)
            item_name_stripped = item_name.strip()
            label_id_to_raw_name[int(item_id)] = item_name_stripped
            raw_class_names.append(item_name_stripped)

# Create a list of unique class names and a mapping from unique name to 0-indexed integer ID
unique_class_names = []
seen_names = set()
for name in raw_class_names:
    if name not in seen_names:
        unique_class_names.append(name)
        seen_names.add(name)

# This will be our canonical list of class names for classification
class_names = unique_class_names
num_unique_classes = len(class_names)

# Create a mapping from label ID (from TFRecord) to our 0-indexed unique class ID
# This assumes that if multiple label_ids map to the same raw_name, they should all map to the same unique_class_idx
label_id_to_unique_class_idx = {}
for label_id, raw_name in label_id_to_raw_name.items():
    if raw_name in class_names: # Should always be true if constructed correctly
        label_id_to_unique_class_idx[label_id] = class_names.index(raw_name)
    else:
        # Fallback or error handling if a raw_name is not in unique_class_names (should not happen)
        label_id_to_unique_class_idx[label_id] = -1 # Indicate an error or unknown class

print("Detected Unique Class Names:", class_names)
print("Number of Unique Classes:", num_unique_classes)
print("Label ID from TFRecord to 0-indexed Unique Class ID Mapping:", label_id_to_unique_class_idx)
print("0-indexed Unique Class ID to Name Mapping:", {idx: name for idx, name in enumerate(class_names)})

# Global variable for downstream use
num_classes = num_unique_classes

# Create a TensorFlow lookup table for mapping raw_label_id to unique_class_idx
# This will replace tf.py_function for robustness within the tf.data pipeline
keys_tensor = tf.constant(list(label_id_to_unique_class_idx.keys()), dtype=tf.int64)
values_tensor = tf.constant(list(label_id_to_unique_class_idx.values()), dtype=tf.int64)
initializer = tf.lookup.KeyValueTensorInitializer(keys_tensor, values_tensor)
label_lookup_table = tf.lookup.StaticHashTable(initializer, default_value=-1)


Detected Unique Class Names: ['Anesthesia Machine', 'BI', 'C-arm', 'Desf', 'ECG', 'ESU', 'EtO', 'MSV', 'US', 'VM', 'X-Ray']
Number of Unique Classes: 11
Label ID from TFRecord to 0-indexed Unique Class ID Mapping: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10}
0-indexed Unique Class ID to Name Mapping: {0: 'Anesthesia Machine', 1: 'BI', 2: 'C-arm', 3: 'Desf', 4: 'ECG', 5: 'ESU', 6: 'EtO', 7: 'MSV', 8: 'US', 9: 'VM', 10: 'X-Ray'}


In [ ]:
import re

# Define the path to a TFRecord file (e.g., from the 'train' split)
tfrecord_file_path = os.path.join(Ruta_general, 'train', 'medical-euipment-detection.tfrecord')

# A parsing function for TFRecord examples
def parse_tf_example(example_proto):
    # Define the features expected in your TFRecord file
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
    }

    example = tf.io.parse_single_example(example_proto, feature_description)

    # Renamed 'image' to 'img_tensor' to avoid shadowing the imported 'image' module
    img_tensor = tf.image.decode_jpeg(example['image/encoded'], channels=3)
    img_tensor = tf.image.resize(img_tensor, [224, 224])
    img_tensor = tf.cast(img_tensor, tf.float32)

    labels_tensor = example['image/object/class/label'].values

    # Check if there are any labels. If not, assign a sentinel value for filtering.
    if tf.size(labels_tensor) > 0:
        raw_label_id = labels_tensor[0]
        # Use the TensorFlow lookup table instead of tf.py_function
        unique_class_idx = label_lookup_table.lookup(raw_label_id)
    else:
        # If no labels, assign -1 to indicate it should be filtered later
        unique_class_idx = tf.constant(-1, dtype=tf.int64)

    return img_tensor, unique_class_idx

# Create a TFRecordDataset
raw_dataset = tf.data.TFRecordDataset(tfrecord_file_path)

# Map the parsing function to the dataset
parsed_dataset_with_sentinel = raw_dataset.map(parse_tf_example)

# Filter out examples that had no labels (indicated by unique_class_idx == -1)
parsed_dataset = parsed_dataset_with_sentinel.filter(lambda image, label: tf.not_equal(label, -1))

# Iterate over a few examples to see the output (for demonstration)
print("\nLoading a few samples from the TFRecord dataset:")
# For printing, we need the 0-indexed ID to name mapping, which can be created from class_names
class_idx_to_name_map_for_print = {idx: name for idx, name in enumerate(class_names)}
for image, unique_class_idx in parsed_dataset.take(3):
    print(f"  Image shape: {image.shape}, Unique Class ID: {unique_class_idx.numpy()} (Class: {class_idx_to_name_map_for_print.get(unique_class_idx.numpy(), 'Unknown')})")

# The `parsed_dataset` now yields (image, unique_class_idx) pairs.
# This dataset can be used for feature extraction and model training.



Loading a few samples from the TFRecord dataset:
  Image shape: (224, 224, 3), Unique Class ID: 8 (Class: US)
  Image shape: (224, 224, 3), Unique Class ID: 9 (Class: VM)
  Image shape: (224, 224, 3), Unique Class ID: 3 (Class: Desf)


#Inicio de modelo de extracción de caracteristicas

In [ ]:
# Model initializations for feature extraction
# include_top=False removes the classification layers, and pooling='avg' provides a feature vector
model_inception = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')
model_resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')
model_vgg = VGG16(weights='imagenet', include_top=False, pooling='avg')

# The previous definitions of Ruta_boot, Ruta_shoe, Ruta_sandal and their usage to collect routes
# are not compatible with the TFRecord dataset structure and have been removed.
# Feature extraction will now directly use the parsed TFRecord dataset.


In [ ]:
# Define batch size
BATCH_SIZE = 32

# Function to apply VGG16 preprocessing
def apply_vgg16_preprocessing_and_label_mapping(image, unique_class_idx):
    image = vgg16_preprocess(image)
    # unique_class_idx is already mapped to the 0-indexed ID in parse_tf_example
    # So we just pass it through
    return image, unique_class_idx

# Prepare the dataset
# Apply preprocessing, shuffle, batch, and prefetch
# Use the parsed_dataset that now yields (image, unique_class_idx)
train_dataset = parsed_dataset.map(apply_vgg16_preprocessing_and_label_mapping)

# The labels are already 0-indexed unique class IDs, suitable for SparseCategoricalCrossentropy.
# No need for one-hot encoding if using SparseCategoricalCrossentropy.

# Shuffle, batch, and prefetch
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(f"Dataset prepared with batch size: {BATCH_SIZE}")
# num_classes is already set globally from cell 8851c084 after ensuring uniqueness
print(f"Number of unique classes detected: {num_classes}")

# You can iterate a batch to see the shapes
for images, labels in train_dataset.take(1):
    print(f"Shape of one batch of images: {images.shape}")
    print(f"Shape of one batch of labels: {labels.shape}")
    print(f"Example of sparse integer label: {labels[0].numpy()}")
    # Verify label is within range [0, num_classes-1]
    if num_classes > 0 and (labels[0].numpy() >= num_classes or labels[0].numpy() < 0):
        print(f"Warning: Label {labels[0].numpy()} is out of range for {num_classes} classes.")
    elif num_classes == 0:
        print("Warning: No unique classes detected, labels might be invalid.")


Dataset prepared with batch size: 32
Number of unique classes detected: 11
Shape of one batch of images: (32, 224, 224, 3)
Shape of one batch of labels: (32,)
Example of sparse integer label: 9


In [ ]:
# Initialize lists to store features and labels
all_features = []
all_labels = []

print("Extracting features from the dataset using VGG16...")

# Iterate over the preprocessed dataset and extract features
# Use 'model_vgg' (VGG16 without top layers) for feature extraction
for images, labels in tqdm(train_dataset):
    # Extract features using the VGG16 model (pooling='avg' already applied)
    features = model_vgg.predict(images, verbose=0)

    # Extend the lists with extracted features and labels
    all_features.extend(features)
    all_labels.extend(labels.numpy())

# Convert lists to NumPy arrays
all_features = np.array(all_features)
all_labels = np.array(all_labels)

print(f"Feature extraction complete. Total samples: {len(all_labels)}")
print(f"Shape of all_features: {all_features.shape}")
print(f"Shape of all_labels: {all_labels.shape}")


Extracting features from the dataset using VGG16...


0it [00:00, ?it/s]WARNING:tensorflow:5 out of the last 25 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x78643591bf60> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
22it [09:21, 25.54s/it]

Feature extraction complete. Total samples: 684
Shape of all_features: (684, 512)
Shape of all_labels: (684,)


In [ ]:
print("Training and evaluating classifier...")

# Initialize lists to store metrics across folds
accuracies = []
precisions = []
recalls = []
f1s = []
kappas = []

# Define the classifier
# Using LinearSVC as it's often a strong baseline for dense features
clf = LinearSVC(random_state=42, dual=True, max_iter=2000)
# Alternatively, for multi-class, LogisticRegression is also good:
# clf = LogisticRegression(random_state=42, multi_class='ovr', solver='liblinear', max_iter=1000)
# Or RandomForestClassifier
# clf = RandomForestClassifier(random_state=42)

# Perform Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for fold, (train_index, test_index) in enumerate(skf.split(all_features, all_labels)):
    print(f"\n--- Fold {fold+1} ---")
    X_train, X_test = all_features[train_index], all_features[test_index]
    y_train, y_test = all_labels[train_index], all_labels[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average='macro', zero_division=0)
    recall = metrics.recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1 = metrics.f1_score(y_test, y_pred, average='macro', zero_division=0)
    kappa = metrics.cohen_kappa_score(y_test, y_pred)

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    kappas.append(kappa)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (macro): {precision:.4f}")
    print(f"Recall (macro): {recall:.4f}")
    print(f"F1-Score (macro): {f1:.4f}")
    print(f"Kappa: {kappa:.4f}")

# Summarize results
print("\n--- Cross-Validation Results ---")
print(f"Mean Accuracy: {np.mean(accuracies):.4f} (Std: {np.std(accuracies):.4f})")
print(f"Mean Precision: {np.mean(precisions):.4f} (Std: {np.std(precisions):.4f})")
print(f"Mean Recall: {np.mean(recalls):.4f} (Std: {np.std(recalls):.4f})")
print(f"Mean F1-Score: {np.mean(f1s):.4f} (Std: {np.std(f1s):.4f})")
print(f"Mean Kappa: {np.mean(kappas):.4f} (Std: {np.std(kappas):.4f})")

# Re-train classifier on the full dataset for deployment
final_clf = LinearSVC(random_state=42, dual=True, max_iter=2000)
final_clf.fit(all_features, all_labels)
print("Classifier trained on full dataset.")


Training and evaluating classifier...

--- Fold 1 ---


Liblinear failed to converge, increase the number of iterations.


Accuracy: 0.8116
Precision (macro): 0.8479
Recall (macro): 0.7901
F1-Score (macro): 0.7967
Kappa: 0.7868

--- Fold 2 ---


Liblinear failed to converge, increase the number of iterations.


Accuracy: 0.8261
Precision (macro): 0.8552
Recall (macro): 0.7924
F1-Score (macro): 0.7975
Kappa: 0.8031

--- Fold 3 ---


...

Liblinear failed to converge, increase the number of iterations.


Accuracy: 0.8382
Precision (macro): 0.7631
Recall (macro): 0.7843
F1-Score (macro): 0.7719
Kappa: 0.8182

--- Fold 10 ---


Liblinear failed to converge, increase the number of iterations.


Accuracy: 0.8824
Precision (macro): 0.9013
Recall (macro): 0.8998
F1-Score (macro): 0.8945
Kappa: 0.8674

--- Cross-Validation Results ---
Mean Accuracy: 0.8320 (Std: 0.0346)
Mean Precision: 0.8387 (Std: 0.0498)
Mean Recall: 0.8212 (Std: 0.0584)
Mean F1-Score: 0.8170 (Std: 0.0535)
Mean Kappa: 0.8108 (Std: 0.0389)
Classifier trained on full dataset.


Liblinear failed to converge, increase the number of iterations.


In [ ]:
# Define the path to save the classifier
classifier_save_path = '/content/drive/MyDrive/A - FPDI/medical_equipment_classifier.joblib'

# Create directory if it doesn't exist
output_dir = os.path.dirname(classifier_save_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the trained classifier
joblib.dump(final_clf, classifier_save_path)
print(f"Classifier saved to: {classifier_save_path}")


Classifier saved to: /content/drive/MyDrive/A - FPDI/medical_equipment_classifier.joblib


#Puesta en marcha del modelo

In [61]:
# Load the pre-trained VGG16 model for feature extraction
# Ensure it's the same configuration as used for training (include_top=False, pooling='avg')
model_vgg_inference = VGG16(weights='imagenet', include_top=False, pooling='avg')

# Load the trained classifier
classifier_load_path = '/content/drive/MyDrive/A - FPDI/medical_equipment_classifier.joblib'
loaded_clf = joblib.load(classifier_load_path)
print(f"Classifier loaded from: {classifier_load_path}")

# Define the path to the new image for prediction
# You can change this to any image you want to classify
new_image_path = '/content/Radiopaco.png'

# Function to preprocess an image and extract features
def extract_features_single_image(img_path, feature_extractor_model, preprocess_func):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0) # Add batch dimension
    x = preprocess_func(x)
    features = feature_extractor_model.predict(x, verbose=0)
    return features.flatten()

# Extract features from the new image
print(f"Extracting features for image: {new_image_path}")
new_image_features = extract_features_single_image(new_image_path, model_vgg_inference, vgg16_preprocess)

# Reshape for prediction if necessary (sklearn expects 2D array for single sample)
new_image_features = new_image_features.reshape(1, -1)

# Make a prediction
predicted_label_id = loaded_clf.predict(new_image_features)[0]

# Retrieve the class name using the global class_names list
# Ensure class_names is available globally from the label map parsing step
if 0 <= predicted_label_id < len(class_names):
    predicted_class_name = class_names[predicted_label_id]
    print(f"Predicted Class ID: {predicted_label_id}, Predicted Class Name: {predicted_class_name}")
else:
    print(f"Predicted Class ID: {predicted_label_id}, Class name not found or invalid ID.")

print("Prediction complete.")

Classifier loaded from: /content/drive/MyDrive/A - FPDI/medical_equipment_classifier.joblib
Extracting features for image: /content/Radiopaco.png
Predicted Class ID: 4, Predicted Class Name: ECG
Prediction complete.


In [60]:
# Define the path to save the classifier
classifier_save_path = '/content/drive/MyDrive/A - FPDI/medical_equipment_classifier_v3.joblib' # Changed path to avoid overwriting image

# Create directory if it doesn't exist
output_dir = os.path.dirname(classifier_save_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the trained classifier
joblib.dump(final_clf, classifier_save_path)
print(f"Classifier saved to: {classifier_save_path}")

Classifier saved to: /content/drive/MyDrive/A - FPDI/medical_equipment_classifier_v3.joblib


In [64]:
# Load the pre-trained VGG16 model for feature extraction
# Ensure it's the same configuration as used for training (include_top=False, pooling='avg')
model_vgg_inference = VGG16(weights='imagenet', include_top=False, pooling='avg')

# Load the trained classifier
classifier_load_path = '/content/drive/MyDrive/A - FPDI/medical_equipment_classifier_v3.joblib' # Ensure this matches the new save path
loaded_clf = joblib.load(classifier_load_path)
print(f"Classifier loaded from: {classifier_load_path}")

# Define the path to the new image for prediction
# YOU MUST CHANGE THIS TO THE ACTUAL PATH OF YOUR IMAGE FILE
new_image_path = '/content/desf.webp' # <--- UPDATE THIS LINE

# Function to preprocess an image and extract features
def extract_features_single_image(img_path, feature_extractor_model, preprocess_func):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0) # Add batch dimension
    x = preprocess_func(x)
    features = feature_extractor_model.predict(x, verbose=0)
    return features.flatten()

# Extract features from the new image
print(f"Extracting features for image: {new_image_path}")
new_image_features = extract_features_single_image(new_image_path, model_vgg_inference, vgg16_preprocess)

# Reshape for prediction if necessary (sklearn expects 2D array for single sample)
new_image_features = new_image_features.reshape(1, -1)

# Make a prediction
predicted_label_id = loaded_clf.predict(new_image_features)[0]

# Get decision scores for all classes
decision_scores = loaded_clf.decision_function(new_image_features)[0] # Get scores for the single sample

# Convert decision scores to "probabilities" using softmax
# This gives a confidence-like score, but it's not a true probability from SVC
confidences = softmax(decision_scores)

# Get the confidence for the predicted class
predicted_confidence = confidences[predicted_label_id] * 100 # Convert to percentage

# Retrieve the class name using the global class_names list
# Ensure class_names is available globally from the label map parsing step
if 0 <= predicted_label_id < len(class_names):
    predicted_class_name = class_names[predicted_label_id]
    print(f"Predicted Class ID: {predicted_label_id}, Predicted Class Name: {predicted_class_name}")
    print(f"Confidence (approximate): {predicted_confidence:.2f}%")
else:
    print(f"Predicted Class ID: {predicted_label_id}, Class name not found or invalid ID.")

print("Prediction complete.")


Classifier loaded from: /content/drive/MyDrive/A - FPDI/medical_equipment_classifier_v3.joblib
Extracting features for image: /content/desf.webp


Predicted Class ID: 3, Predicted Class Name: Desf
Confidence (approximate): 72.66%
Prediction complete.


### Investigating Label Distribution

Before proceeding with cross-validation, let's examine the distribution of classes in your dataset. This will help understand why `StratifiedKFold` might be failing, especially if some classes have very few samples, making it impossible to distribute them across all folds evenly.

Importar libreria desde Roboflow, dataset de 1029 imagenes

Once you see the output of the directory structure, you'll likely find folders like `train`, `valid`, and `test`, each containing `.tfrecord` files (e.g., `_annotations.tfrecord`). To load these, you'll typically use TensorFlow's `tf.data.TFRecordDataset` API, which is designed to parse these files and extract images and labels. You would then need to adapt your feature extraction and classification pipeline to work with this new data loading mechanism, rather than directly reading image files from individual category folders.

### 1. Extracting Class Names from the Label Map

First, we'll parse the `medical-euipment-detection_label_map.pbtxt` file to get the mapping of numerical IDs to class names. This is crucial for understanding what each label in the TFRecord files refers to.

### 2. Reading and Parsing TFRecord Files

Now that we have the class names, the next step is to read the `.tfrecord` files. This involves using `tf.data.TFRecordDataset` and a parsing function to extract images and their corresponding label IDs from each record. Below is an example of how you might set up a basic parsing function and load a TFRecord file.

**Note:** The exact feature names within the `feature_description` (e.g., `'image/encoded'`, `'image/object/class/label'`) can vary based on how the TFRecord was generated. You might need to inspect one of your `.tfrecord` files or refer to the Roboflow dataset's documentation for the precise feature keys.

After downloading, the dataset will be available in the specified `dataset.location` directory. You can then load images and annotations from this directory based on the format you chose (e.g., 'tensorflow', 'yolov5').

## Preparación del Dataset para Entrenamiento

Ahora que hemos parseado correctamente los TFRecords y tenemos un mapeo de IDs a nombres de clases, el siguiente paso es preparar el dataset para el entrenamiento. Esto implica:

1.  **Crear lotes (batches)**: Agrupar múltiples imágenes para que el modelo pueda procesarlas eficientemente.
2.  **Aplicar preprocesamiento**: Usar la función `preprocess_input` del modelo VGG16 para escalar y normalizar los valores de los píxeles de la imagen de acuerdo a los requisitos del modelo.
3.  **Configurar la precarga (prefetch)**: Optimizar el rendimiento de la carga de datos para que el CPU prepare el siguiente lote mientras el GPU procesa el actual.

# Task
Create a pipeline to classify images as "Boot", "Shoe", or "Sandal" using pre-trained deep learning models (VGG16, ResNet50, InceptionResNetV2) for feature extraction and a Multinomial Naive Bayes classifier for classification. The process involves:
1.  **Initializing Libraries and Loading Models**: Set up necessary libraries and load the pre-trained VGG16, ResNet50, and InceptionResNetV2 models.
2.  **Organizing and Labeling Data**: Collect image paths for each category ("Boot", "Shoe", "Sandal") from the dataset located at `/content/drive/MyDrive/A - FPDI/Shoe vs Sandal vs Boot Dataset`, and create label files for binary classification scenarios (Boot vs Shoe, Boot vs Sandal, Shoe vs Sandal).
3.  **Extracting Features**: Implement a function to extract features from images using the VGG16 model and save these features to text files.
4.  **Training and Evaluating the Classifier**: Read the extracted features and labels, perform stratified cross-validation, train a Multinomial Naive Bayes classifier, and evaluate its performance using metrics like accuracy, precision, recall, and F1-score.
5.  **Saving the Classifier**: Save the trained classifier using `joblib`.
6.  **Predicting with a New Image**: Load a new image (e.g., `/content/drive/MyDrive/A - FPDI/Bota.jpg`), extract its features using the VGG16 model, load the saved classifier, and make a prediction on the image's category.

## Inicialización y Carga de Modelos

### Subtask:
Initialize necessary libraries, load pre-trained deep learning models (VGG16, ResNet50, InceptionResNetV2), and define file paths for the dataset.


## Finalizar Tarea

### Subtask:
Resume el flujo general del código, desde la entrada de datos hasta la salida de predicciones, resaltando los componentes principales y su interacción.


## Summary:

### Data Analysis Key Findings

*   All required libraries, including `matplotlib.pyplot`, `numpy`, `cv2`, `tensorflow`, `keras`, `os`, and specific Keras application modules, were successfully imported.
*   Pre-trained deep learning models (`VGG16`, `ResNet50`, and `InceptionResNetV2`) were confirmed to be loaded with 'imagenet' weights.
*   File paths for the dataset, including specific sub-paths for 'Boot', 'Shoe', and 'Sandal' categories, and output directories for feature extraction, were correctly defined.

### Insights or Next Steps

*   The successful initialization of libraries and loading of models forms a robust foundation for the subsequent data processing and model training steps, ensuring that all necessary tools are in place.
*   The clear definition of file paths is crucial for organized data access and feature storage, which will streamline the workflow for feature extraction and model training.
